In [4]:
#Required Libraries


import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [78]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [97]:
 nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...


True

In [100]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...


True

In [79]:
# data gathering
df = pd.read_csv("C:\\Users\\Administrator\\Desktop\\Fake News Classification\\WELFake_Dataset.csv")
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72134 non-null  int64 
 1   title       71576 non-null  object
 2   text        72095 non-null  object
 3   label       72134 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB


In [81]:
df["label"].value_counts()

1    37106
0    35028
Name: label, dtype: int64

In [82]:
rows, coulumns = df.shape

In [83]:
rows, coulumns

(72134, 4)

In [84]:
df.isna().sum()

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [85]:
df = df.dropna()
df.shape

(71537, 4)

In [86]:
df.reset_index(inplace=True)
df.head()

,index,Unnamed: 0,title,text,label
0,0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
2,3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
3,4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
4,5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


In [87]:
df['title'][0]

'LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]'

In [88]:
df = df.drop(["Unnamed: 0","text"], axis= 1)
df.head()

,index,title,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1
1,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1
2,3,"Bobby Jindal, raised Hindu, uses story of Chri...",0
3,4,SATAN 2: Russia unvelis an image of its terrif...,1
4,5,About Time! Christian Group Sues Amazon and SP...,1


# Data preprocessing

# 01. Tokenization

In [89]:
sample_data = 'SATAN 2 Russia unvelis an image of its terrif'
sample_data = sample_data.split()
sample_data

['SATAN', '2', 'Russia', 'unvelis', 'an', 'image', 'of', 'its', 'terrif']

# 02. Lowercase

In [90]:
sample_data = [data.lower() for data in sample_data]
sample_data

['satan', '2', 'russia', 'unvelis', 'an', 'image', 'of', 'its', 'terrif']

# 03. Remove Stopwords

In [121]:
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords[0:10])
print(len(stopwords))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]
179


In [122]:
#sample_data = 'SATAN 2 Russia unvelis an image of its terrif'
sample_data = [data for data in sample_data if data not in stopwords]
sample_data

['satan', '2', 'russia', 'unvelis', 'image', 'terrif']

# 04. Stemming

In [129]:
ps = PorterStemmer()
sample_data_stemming = [ps.stem(data) for data in sample_data]
print(sample_data_stemming)

['satan', '2', 'russia', 'unv', 'imag', 'terrif']


# 05. Lemmatization

In [124]:
lm = WordNetLemmatizer()
sample_data_lemma = [lm.lemmatize(data) for data in sample_data]
print(sample_data_lemma)
print("hi")

['satan', '2', 'russia', 'unvelis', 'image', 'terrif']
hi


In [127]:
lm = WordNetLemmatizer()
corpus = []        # dataset in NLP
for i in range(len(df)):
    review = re.sub('A-Za-z0-9',' ', df['title'][i])
    review = review.lower()
    review = review.split()
    review = [lm.lemmatize(i) for i in review if i not in stopwords]
    review = " ".join(review)
    corpus.append(review)


In [128]:
len(corpus)

71537

# vectorization

In [136]:
tf = TfidfVectorizer()
del corpus[len(corpus)//4:]
x = tf.fit_transform(corpus).toarray()
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [151]:
y = df['label'][:len(df['label'])//8]
y.head()

0    1
1    1
2    0
3    1
4    1
Name: label, dtype: int64

In [152]:
len(df['label'])//8

8942

In [160]:
len(y)

8942

# Train and Test dataset

In [154]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state= 10, stratify = y)
# stratify ==> dividing a dataset into subgroups

# model building

In [162]:
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

RandomForestClassifier()

# model evaluation

In [171]:
from sklearn.metrics import accuracy_score
y_pred = rf.predict(x_test)
accuracy_score_ = accuracy_score(y_test,y_pred) 
accuracy_score_

0.8486768542676109

In [183]:
class Evaluation:
    def __init__(self, model,x_train, x_test, y_train, y_test):
        self.model = model
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test
    def train_evaluation(self):
        y_pred = self.model.predict(self.x_train)
        acc_scr_train = accuracy_score(self.y_train, y_pred)
        print("Accuracy score on training Dataset : ",acc_scr_train)
        
        con_mat_train = confusion_matrix(self.y_train, y_pred)
        print("Confusion matrix on training Dataset : \n", con_mat_train)
        
        class_rep_train = classification_report(self.y_train, y_pred)
        print("classification report on training Dataset : \n", class_rep_train)
        
    def test_evaluation(self):
        y_pred = self.model.predict(self.x_test)
        acc_scr_test = accuracy_score(self.y_test, y_pred)
        print("Accuracy score on testing Dataset : ",acc_scr_test)
        
        con_mat_test = confusion_matrix(self.y_test, y_pred)
        print("Confusion matrix on testing Dataset : \n", con_mat_test)
        
        class_rep_test = classification_report(self.y_test, y_pred)
        print("classification report on testing Dataset : \n", class_rep_test)
    
    
    


# checking the accuracy of training dataset

In [184]:
Evaluation(rf,x_train, x_test, y_train, y_test).train_evaluation()

Accuracy score on training Dataset :  1.0
Confusion matrix on training Dataset : 
 [[2994    0]
 [   0 3265]]
classification report on training Dataset : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2994
           1       1.00      1.00      1.00      3265

    accuracy                           1.00      6259
   macro avg       1.00      1.00      1.00      6259
weighted avg       1.00      1.00      1.00      6259



# checking accuracy on testing dataset

In [185]:
Evaluation(rf,x_train, x_test, y_train, y_test).test_evaluation()

Accuracy score on testing Dataset :  0.8486768542676109
Confusion matrix on testing Dataset : 
 [[1027  257]
 [ 149 1250]]
classification report on testing Dataset : 
               precision    recall  f1-score   support

           0       0.87      0.80      0.83      1284
           1       0.83      0.89      0.86      1399

    accuracy                           0.85      2683
   macro avg       0.85      0.85      0.85      2683
weighted avg       0.85      0.85      0.85      2683



# prediction pipeline

In [213]:
class preprocessing:
    def __init__(self, data):
        self.data = data
    def text_preprocessing_user(self):
        lm = WordNetLemmatizer()
        pred_data = [self.data]
        preprocess_data = []
        for data in pred_data:
            review = re.sub('A-Za-z0-9',' ',data)
            review = review.lower()
            review = review.split()
            review = [lm.lemmatize(i) for i in review if i not in stopwords]
            review = ' '.join(review)
            preprocess_data.append(review)
        return preprocess_data

In [222]:
class Prediction:
    def __init__(self,pred_data, model):
        self.pred_data = pred_data
        self.model = model
    
    def prediction_model(self):
        preprocessed_data = preprocessing(self.pred_data).text_preprocessing_user()
        data = tf.transform(preprocessed_data)
        prediction = self.model.predict(data)
        
        if prediction[0] == 0:
            return "The news is fake"
        else:
            return "The news is Real"
            

In [233]:
df["title"][2]

'Bobby Jindal, raised Hindu, uses story of Christian conversion to woo evangelicals for potential 2016 bid'

In [234]:
data = 'Bobby Jindal, raised Hindu, uses story of Christian conversion to woo evangelicals for potential 2016 bid'
preprocessing(data).text_preprocessing_user()

['bobby jindal, raised hindu, us story christian conversion woo evangelicals potential 2016 bid']

In [240]:
Prediction(data,rf).prediction_model()

'The news is Real'